# **Clase 14**
###### *Miercoles 3 de marzo del 2021*

---

## ***Metodos de optimización***

El objetivo principal de los métodos de optimización en deep learning es minimizar la función objetivo, que regularmente es el promedio de la función de pérdida. Esto nos da:

$$
f(x) = \frac{1}{n} \sum \limits _{i=1} ^{n} f_{i}(x) \ .
$$

Mientras que el gradiente de la función objetivo está dada por el vector de la derivada de la función objetivo con respecto a cada una de las variables:

$$
\nabla f(x) = \begin{bmatrix} \frac{\partial f(x)}{\partial x_{1}} \\ : \\ \frac{\partial f(x)}{\partial x_{p}} \end{bmatrix} = \frac{1}{n} \sum \limits _{i=1} ^{n} \nabla f_{i}(x) \ .
$$

Donde *p* es el número de variables de entrada. Una vez calculado el gradiente, la funcion de actualización de cada variable esta dada por:

$$
x \leftarrow x - \eta \nabla f(x) \ .
$$

Donde $\eta$ es la tasa de aprendizaje. Por lo tanto, el costo computacional de cada iteración sería de $O(n)$. Eso implica que la complejidad crezca junto con el conjunto de datos de entrenamiento. Entre más grande sea, el costo de cada iteración sera más grande. 

### ***Stochastic Gradient Descent (SGD)***

SGD reduce el costo computacional al seleccionar uniformemente de manera aleatoria un valor $i \in \{1,..., n\}$. Se reduce el costo computacional de $O(n)$ a $O(1)$. 
 
$$
\tilde{\nabla} f(x) = \nabla f_{i}(x) \ .
$$

De esta manera la actualización de $x$ está dada por:

$$
x \leftarrow x - \eta \nabla f_{i}(x) \ .
$$

### ***B-SGD***

El hecho solo seleccionar uno de los gradientes puede hacer que nos perdamos información y el algoritmo no sea completamente estable. Para eso podemos usar un subgrupo $B$ de los gradientes tal que $B \subseteq \{1,...,n\}$ y $|B| \leq n$. El gradiente estaría dado por:

$$
\nabla_{B} f(x) = \frac{1}{|B|} \sum \limits _{i \in B} ^{} \nabla f_{i}(x) \ .
$$

Deseablemente se espera que el tamaño de $B$ sea muy grande. Pero nos enfrentamos al mismo problema, el costo computacional. De tal manera que se busca que $B$ sea lo más grande posible sin afectar el costo computacional. 

### ***Métodos acelerados (Inercia)***

Con el método anterior podemos utilizar un subconjunto de los valores para calcular el gradiente. Eso ayuda a mitigar la varianza y optimizar el costo computacional. Los métodos acelerados ofrecen no solo usar el promedio actual, sino el promedio acumulado de iteraciones pasadas. 

$$
V_{t} \leftarrow \beta V_{t-1} + g_{t, t-1} \ .
$$

Donde

$$
g_{t, t-1} = \tilde{\nabla}_{B} f(x_{t-1}) \ .
$$

Y $\beta \in \{0, 1\}$. De esta manera, $V$ va acumulando gradientes pasados. Si expandimos $V_{t}$

$$
V_{t} = \beta^2 V_{t-2} + \beta g_{t-1, t-2} + g_{t, t-1} = \sum \limits _{k=0} ^{t-1} \beta ^k g_{t-k, t-k-1} \ .
$$

Para este caso, la actualización ya no se haría únicamente con gradiente. Las ecuaciones de actualización quedarían:

$$
V_{t} = \beta V_{t-1} + g_{t,t-1} \ .
$$
$$
x_{t} = x_{t-1} - \eta V_{t} \ .
$$
Esto nos deja que si $B \approx 1$ el promedio es un promedio largo. \
Mientras que si $B \approx 0$ la corrección al gradiente es pequeña. De tal manera que antes los pasos en cada iteración estaban en función de la tasa de aprendizaje $\eta$, ahora los pasos son de tamaño $\frac{\eta}{1-\beta}$.

### ***Adagrad***

Una función de pérdida puede tener distinta sensibilidad en diferentes componentes. Esto debido a que puede haber casos donde ciertos escenarios ocurran con menos frecuencia. Por ejemplo, en el entrenamiento de algún modelo de lenguaje pueden existir palabras más frecuentes que otras, y las palabras frecuentes pueden ir mejorando y beneficiándose de eso, mientras que las palabras menos frecuentes se estancan. \

Derivado del problema de minimizar la funcion:

$$
f(x) = \frac{1}{2} x^T Qx + c^T x + b  \ .
$$

Podemos reescribirlo en términos de su descomposición propia $Q = U^T \Lambda U$

$$
\bar{f}(\bar{x}) = \frac{1}{2} \bar{x}^T \Lambda \bar{x} + \bar{c}^T \bar{x} + b \ .
$$

Con 

$$
\bar{x} = U^T x \ .
$$

Derivado de 

$$
x^T Q x = x^T U \Lambda U^T x \ .
$$

Debido a que calcular los eigenvalores exactos puede ser caro, aproximarlos puede ser una alternativa. Para esto, tomamos la diagonal de $Q$ y la reescalamos. 

$$
\tilde{Q} =diag^{-\frac{1}{2}}(Q) Q diag^{-\frac{1}{2}}(Q) \ .
$$

El resultado de esto es que cada componente $\tilde{Q}_{ij} = \frac{Q_{ij}}{\sqrt{Q_{ii}Q_{jj}}}$ y ${Q}_{ii} = 1$ para todo $i$ \

De igual manera es complicado acceder a la segunda derivada de la función objetivo, ya que computacionalmente es muy costoso. Adagrad usa un proxy para evitar esto, sacando provecho de la magnitud del mismo gradiente.

$$
\nabla f(x) = \Lambda \bar{x} + \bar{c} \ .
$$

Por lo tanto, esto nos lleva a las siguientes ecuaciones de actualización. Usaremos $S_{t}$ para acumular la varianza de gradientes pasados. 

$$
g_{t} = \tilde{\nabla}_{B_{t}} f(x_{t-1}) \ .
$$
$$
S_{t} = S_{t-1} + G^2_{t} \ .
$$
$$
x_{t} = x_{t-1} - \eta \ diag(S_{t}+ \epsilon)^{-\frac{1}{2}} g_{t} \ .
$$

### ***RMSProp***

El método Adagrad funciona bien para funciones convexas, pero para funciones no convexas. El problema radica en que se acumulan los cuadrados de los gradientes en la variable $S_{t}$, lo que provoca que vaya creciendo sin límite debido a que no existe una normalización. El método RMSProp agrega un parámetro $\gamma$ para controlar el crecimiento, dejando todo lo demás intacto. 

$$
S_{t} = \gamma S_{t-1} + (1-\gamma) \ G^2_{t} \ .
$$
$$
x_{t} = x_{t-1} - \eta \ diag(S_{t}+ \epsilon)^{-\frac{1}{2}} g_{t} \ .
$$

Si expandimos la definición de $S_{t}$

$$
S_{t} = (1 - \gamma) G^2_{t} + \gamma S_{t-1} = (1 - \gamma) \ (G^2_{t} + \gamma G^2_{t-1} + ...) \ .
$$

De aquí tenemos que 
$$
\sum \gamma^k = \frac{1}{1-\gamma} \ .
$$
Por lo tanto la suma de pesos está normalizado a 1. 

### ***Adadelta***

Adadelta es otra variante de *Adagrad*, muy similar a *RMSProp*. Tenemos dos variables $S_{t}$ y $\nabla x_{t}$, la primera almacena el promedio de los grdadientes, mientras que la segunda almacena el promedio de los cambios de los parámetros. De igual manera que en *RMSProp*, tenemos un parámetro $\gamma$

$$
S_{t} = \gamma S_{t-1} + (1-\gamma) \ G^2_{t} \ .
$$
$$
\nabla x_{t} = \gamma \nabla x_{t-1} + (1-\gamma) \ \hat{g} \odot \hat{g} \ .
$$

Una de las principales diferencias con el método anterior es que las actualizaciones se hacen con el gradiente reescalado $\hat{g}_{t}$

$$
\hat{g}_{t} = diag(\frac{\nabla x_{t-1} + \epsilon}{S_{t}+\epsilon}) \ g_{t} \ .
$$
$$
x_{t} = x_{t-1} - \hat{g}_{t} \ .
$$

### ***Adam***

*Adam* toma un poco de los conceptos de los métodos anteriores y los junta para crear un nuevo método. Se tienen las dos variables de estado $V_{t}$ y $S_{t}$.

$$
V_{t} = \beta_{1} V_{t-1} + (1 - \beta_{1}) \ g_{t} \ .
$$
$$
S_{t} = \beta_{2} S_{t-1} + (1 - \beta_{2}) \ G^2_{t} \ .
$$

Donde $\beta_{1}$ y $\beta_{2}$ son parámetros no negativos de peso. Regularmente se escogen los valores $\beta_{1} = 0.9$ y $\beta_{2} = 0.999$. Las variables de estado normalizadas están dadas por: 

$$
\tilde{V}_{t} = \frac{V_{t}}{1-\beta^t_{1}} \ .
$$
\
$$
\tilde{S}_{t} = \frac{S_{t}}{1-\beta^t_{2}} \ .
$$

Similar a *RMSProp* el gradiente se reescala usando la siguiente ecuación:

$$
\hat{g}_{t} = \eta \frac{\tilde{V}_{t}}{\sqrt{S_{t}}+ \epsilon} \ .
$$

Y por último, la actualización se da con:

$$
x_{t} = x_{t-1} - \hat{g}_t \ .
$$